In [1]:
!pip install -q modelscope open_clip_torch pytorch-lightning


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install -q http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_20_04_release/wheels/openvino-2023.1.0.dev20230801-11888-cp310-cp310-manylinux_2_31_x86_64.whl http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_20_04_release/wheels/openvino_dev-2023.1.0.dev20230801-11888-py3-none-any.whl

  ERROR: HTTP error 404 while getting http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_20_04_release/wheels/openvino-2023.1.0.dev20230801-11888-cp310-cp310-manylinux_2_31_x86_64.whl
ERROR: Could not install requirement openvino==2023.1.0.dev20230801 from http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_20_04_release/wheels/openvino-2023.1.0.dev20230801-11888-cp310-cp310-manylinux_2_31_x86_64.whl because of HTTP error 404 Client Error: Not Found for url: http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_20_04_release/wheels/openvino-2023.1.0.dev20230801-11888-cp310-cp310-manylinux_2_31_x86_64.whl for URL http://10.211.120.125/openvino_ci/private_builds/dldt/master/pre_commit/25707c65c8f3b3de2915e6f4aa0ea485593adf5d/private_linux_ubuntu_2

In [3]:
import os
import functools
from pathlib import Path

from modelscope.pipelines import pipeline
from modelscope.models.multi_modal.video_synthesis.text_to_video_synthesis_model import FrozenOpenCLIPEmbedder
from modelscope.outputs import OutputKeys
import torch
import gradio as gr

from openvino.tools import mo
from openvino.runtime import serialize
from openvino.runtime.ie_api import CompiledModel


torch.load = functools.partial(torch.load, map_location='cpu')
p = pipeline('text-to-video-synthesis', 'damo/text-to-video-synthesis', device='cpu')
p.model.clip_encoder = FrozenOpenCLIPEmbedder(version=os.path.join(p.model.model_dir, p.model.config.model.model_args.ckpt_clip), layer='penultimate', device='cpu')

2023-08-07 21:05:22,363 - modelscope - INFO - PyTorch version 1.13.1+cpu Found.
2023-08-07 21:05:22,370 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-08-07 21:05:22,371 - modelscope - INFO - Loading ast index from /home/itrushkin/.cache/modelscope/ast_indexer
2023-08-07 21:05:22,618 - modelscope - INFO - Loading done! Current index file version is 1.7.1, with md5 e847f1b015df372fdb9573c01b4a727f and a total number of 861 components indexed
2023-08-07 21:05:33,898 - modelscope - INFO - Model revision not specified, use the latest revision: v1.1.0
2023-08-07 21:05:35,637 - modelscope - INFO - initiate model from /home/itrushkin/.cache/modelscope/hub/damo/text-to-video-synthesis
2023-08-07 21:05:35,638 - modelscope - INFO - initiate model from location /home/itrushkin/.cache/modelscope/hub/damo/text-to-video-synthesis.
2023-08-07 21:05:35,644 - modelscope - INFO - initialize model from /home/itrushkin/.cache/modelscope/hub/damo/text-to-video-synthesis
2023-08-07 21:07:24,449

## Conversion via PyTorch

In [7]:
from typing import NamedTuple, Tuple, Optional


class ModelInfo(NamedTuple):
    model: torch.nn.Module
    example_input: torch.Tensor | Tuple[torch.Tensor]
    xml_path: str

    def convert_and_serialize(self):
        if not Path(self.xml_path).exists():
            ov_model = mo.convert_model(self.model, example_input=self.example_input)
            serialize(ov_model, self.xml_path)
        else:
            print(f'{Path(self.xml_path).resolve()} exists.')

    def compile(self, device: Optional[str] = None):
        if not Path(self.xml_path).exists():
            print("Model is not converted yet. Converting...")
            self.convert_and_serialize()
        ov_model = core.read_model(self.xml_path)
        return core.compile_model(ov_model)
        


models = {
    "unet": ModelInfo(
        p.model.sd_model,
        {"x": torch.randn(1, 4, 16, 32, 32), "t": torch.randn(1), "y": torch.randn(1, 77, 1024)},
        "unet/unet.xml",
    ),
    "decoder": ModelInfo(
        p.model.autoencoder.decoder, torch.randn(16, 4, 32, 32), "autoencoder/decoder.xml"
    ),
    "post_quant_conv": ModelInfo(
        p.model.autoencoder.post_quant_conv,
        torch.randn(16, 4, 32, 32),
        "autoencoder/post_quant_conv.xml",
    ),
    "token_embedding": ModelInfo(
        p.model.clip_encoder.model.token_embedding,
        torch.randn(1, 77),
        "clip_encoder/token_embedding.xml",
    ),
    "ln_final": ModelInfo(
        p.model.clip_encoder.model.ln_final,
        torch.randn(1, 77, 1024),
        "clip_encoder/ln_final.xml"
    )
}
for i, r in enumerate(p.model.clip_encoder.model.transformer.resblocks):
    models[f"resblock_{i}"] = ModelInfo(
        r,
        {"q_x": torch.randn(77, 1, 1024), "attn_mask": torch.randn(77, 77)},
        f"clip_encoder/resblock_{i}.xml",
    )
RESBLOCKS_COUNT = len(p.model.clip_encoder.model.transformer.resblocks)

In [8]:
def call(ov_model: CompiledModel, input: torch.Tensor):
    out = ov_model(input)
    out = out[ov_model.outputs[0]]
    out = torch.tensor(out)
    return out

In [9]:
for name, model_info in models.items():
    model_info.convert_and_serialize()

/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/unet/unet.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/autoencoder/decoder.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/autoencoder/post_quant_conv.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/clip_encoder/token_embedding.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/clip_encoder/ln_final.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/clip_encoder/resblock_0.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/clip_encoder/resblock_1.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-damo-text2video/clip_encoder/resblock_2.xml exists.
/home/itrushkin/repos/openvino_notebooks/notebooks/244-modelscope-d

In [10]:
from functools import partial
import numpy as np

def p_mean_variance(self,
                    xt,
                    t,
                    model,
                    model_kwargs={},
                    clamp=None,
                    percentile=None,
                    guide_scale=None):
    r"""Distribution of p(x_{t-1} | x_t).
    """
    def _i(tensor, t, x):
        r"""Index tensor using t and format the output according to x.
        """
        tensor = tensor.to(x.device)
        shape = (x.size(0), ) + (1, ) * (x.ndim - 1)
        return tensor[t].view(shape).to(x)
    
    # predict distribution
    if guide_scale is None:
        
        out = model({"x": xt, "t": self._scale_timesteps(t), **model_kwargs})
    else:
        # classifier-free guidance
        # (model_kwargs[0]: conditional kwargs; model_kwargs[1]: non-conditional kwargs)
        assert isinstance(model_kwargs, list) and len(model_kwargs) == 2
        y_out = call(model, {"x": xt, "t": self._scale_timesteps(t), **model_kwargs[0]})
        u_out = call(model, {"x": xt, "t": self._scale_timesteps(t), **model_kwargs[1]})
        dim = y_out.size(1) if self.var_type.startswith(
                'fixed') else y_out.size(1) // 2
        a = u_out[:, :dim]
        b = guide_scale * (y_out[:, :dim] - u_out[:, :dim])
        c = y_out[:, dim:]
        out = torch.cat([a + b, c], dim=1)

    # compute variance
    if self.var_type == 'fixed_small':
        var = _i(self.posterior_variance, t, xt)
        log_var = _i(self.posterior_log_variance_clipped, t, xt)

    # compute mean and x0
    if self.mean_type == 'eps':
        x0 = _i(self.sqrt_recip_alphas_cumprod, t, xt) * xt - _i(
            self.sqrt_recipm1_alphas_cumprod, t, xt) * out
        mu, _, _ = self.q_posterior_mean_variance(x0, xt, t)

    # restrict the range of x0
    if percentile is not None:
        assert percentile > 0 and percentile <= 1  # e.g., 0.995
        s = torch.quantile(
            x0.flatten(1).abs(), percentile,
            dim=1).clamp_(1.0).view(-1, 1, 1, 1)
        x0 = torch.min(s, torch.max(-s, x0)) / s
    elif clamp is not None:
        x0 = x0.clamp(-clamp, clamp)
    return mu, var, log_var, x0

p.model.diffusion.p_mean_variance = partial(p_mean_variance, p.model.diffusion)

In [11]:
from einops import rearrange

def forward(self, input):
    r"""
    The entry function of text to image synthesis task.
    1. Using diffusion model to generate the video's latent representation.
    2. Using vqgan model (autoencoder) to decode the video's latent representation to visual space.

    Args:
        input (`Dict[Str, Any]`):
            The input of the task
    Returns:
        A generated video (as pytorch tensor).
    """
    y = input['text_emb']
    zero_y = input['text_emb_zero']
    context = torch.cat([zero_y, y], dim=0).to(self.device)
    # synthesis
    num_sample = 1  # here let b = 1
    max_frames = self.config.model.model_args.max_frames
    latent_h, latent_w = input['out_height'] // 8, input[
        'out_width'] // 8
    x0 = self.diffusion.ddim_sample_loop(
        noise=torch.randn(num_sample, 4, max_frames, latent_h,
                          latent_w).to(
                              self.device),  # shape: b c f h w
        model=self.sd_model,
        model_kwargs=[{
            'y':
            context[1].unsqueeze(0).repeat(num_sample, 1, 1)
        }, {
            'y':
            context[0].unsqueeze(0).repeat(num_sample, 1, 1)
        }],
        guide_scale=9.0,
        ddim_timesteps=50,
        eta=0.0)

    scale_factor = 0.18215
    video_data = 1. / scale_factor * x0
    bs_vd = video_data.shape[0]
    video_data = rearrange(video_data, 'b c f h w -> (b f) c h w')
    video_data = self.autoencoder.decode(video_data)
    video_data = rearrange(
        video_data, '(b f) c h w -> b c f h w', b=bs_vd)
    return video_data.type(torch.float32)
p.model.forward = partial(forward, p.model)

In [12]:
class OVAutoencoderKL:
    def __init__(self):
        self.decoder = models['decoder'].compile()
        self.post_quant_conv = models['post_quant_conv'].compile()

    def decode(self, z):
        z = call(self.post_quant_conv, z)
        dec = call(self.decoder, z)
        return dec

In [21]:
import open_clip

class OVFrozenOpenClipEncoder:
    def __init__(self, positional_embedding: torch.Tensor, attn_mask: torch.Tensor):
        self.token_embedding = models['token_embedding'].compile()
        self.transformer_resblocks = [models[f'resblock_{i}'].compile() for i in range(RESBLOCKS_COUNT)]
        self.layer_idx = 1
        self.positional_embedding = positional_embedding
        self.attn_mask = attn_mask
        self.ln_final = models['ln_final'].compile()

    def __call__(self, text):
        tokens = open_clip.tokenize(text)
        z = self.encode_with_transformer(tokens)
        return z

    def encode_with_transformer(self, text):
        x = call(self.token_embedding, text)  # [batch_size, n_ctx, d_model]
        x = x + self.positional_embedding
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.text_transformer_forward(x, attn_mask=self.attn_mask)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = call(self.ln_final, x)
        return x

    def text_transformer_forward(self, x: torch.Tensor, attn_mask=None):
        for i, r in enumerate(self.transformer_resblocks):
            if i == len(self.transformer_resblocks) - self.layer_idx:
                break
            x = call(r, {"q_x": x, "attn_mask": attn_mask})
        return x

In [14]:
from functools import partial
import tempfile
from modelscope.outputs import OutputKeys
import cv2

def preprocess(self, input, **preprocess_params):
    text_emb = self.model.clip_encoder(input['text'])
    text_emb_zero = self.model.clip_encoder('')
    out_height = input['height'] if 'height' in input else 256
    out_width = input['width'] if 'height' in input else 256
    return {
        'text_emb': text_emb,
        'text_emb_zero': text_emb_zero,
        'out_height': out_height,
        'out_width': out_width
    }

# def postprocess(self, inputs, **post_params):
#     video = tensor2vid(inputs['video'])
#     output_video_path = post_params.get('output_video', None)
#     temp_video_file = False
#     if output_video_path is None:
#         output_video_path = tempfile.NamedTemporaryFile(suffix='.mp4').name
#         temp_video_file = True

#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     h, w, c = video[0].shape
#     video_writer = cv2.VideoWriter(
#         output_video_path, fourcc, fps=8, frameSize=(w, h))
#     for i in range(len(video)):
#         img = cv2.cvtColor(video[i], cv2.COLOR_RGB2BGR)
#         video_writer.write(img)
#     video_writer.release()
#     if temp_video_file:
#         video_file_content = b''
#         with open(output_video_path, 'rb') as f:
#             video_file_content = f.read()
#         os.remove(output_video_path)
#         return {OutputKeys.OUTPUT_VIDEO: video_file_content}
#     else:
#         return {OutputKeys.OUTPUT_VIDEO: output_video_path}

p.preprocess = partial(preprocess, p)
# p.postprocess = partial(postprocess, p)

In [15]:
positional_embedding = p.model.clip_encoder.model.positional_embedding
attn_mask = p.model.clip_encoder.model.attn_mask

In [16]:
del p.model.sd_model
del p.model.autoencoder
del p.model.clip_encoder
import gc
gc.collect()

794

In [22]:
from openvino.runtime import Core
core=Core()

p.model.sd_model = models['unet'].compile()
p.model.autoencoder = OVAutoencoderKL()
p.model.clip_encoder = OVFrozenOpenClipEncoder(positional_embedding, attn_mask)

In [23]:
p({'text': 'Little panda eating bamboo'}, output_video='output.mp4')

{'output_video': 'output.mp4'}

## Conversion via ONNX

In [ ]:
# UNET_PATH = Path('unet')
# UNET_PATH.mkdir(exist_ok=True)
# torch.onnx.export(p.model.sd_model, (torch.randn(1, 4, 16, 32, 32),torch.randn(1), torch.randn(1, 77, 1024)), str(UNET_PATH / 'unet.onnx'))
# unet = mo.convert_model(str(UNET_PATH / 'unet.onnx'))
# serialize(unet, str(UNET_PATH / 'unet.xml'))

In [ ]:
# AUTOENCODER_PATH = Path('autoencoder')
# Path.mkdir(AUTOENCODER_PATH, exist_ok=True)
# torch.onnx.export(p.model.autoencoder.decoder, torch.randn(16, 4, 32, 32), str(AUTOENCODER_PATH / 'decoder.onnx'))
# torch.onnx.export(p.model.autoencoder.post_quant_conv, torch.randn(16, 4, 32, 32), str(AUTOENCODER_PATH / 'post_quant_conv.onnx'))

In [ ]:
# CLIP_ENCODER_PATH = Path('clip_encoder')
# CLIP_ENCODER_PATH.mkdir(exist_ok=True)
# torch.onnx.export(p.model.clip_encoder, 'sample text', str(CLIP_ENCODER_PATH / 'clip_encoder.onnx'))

In [ ]:
# ov_clip_encoder = mo.convert_model(str(CLIP_ENCODER_PATH / 'clip_encoder.onnx'))
# serialize(ov_clip_encoder, str(CLIP_ENCODER_PATH / 'clip_encoder.xml'))

In [ ]:
# ov_decoder = mo.convert_model(str(AUTOENCODER_PATH / 'decoder.onnx'))
# serialize(ov_decoder, str(AUTOENCODER_PATH / 'decoder.xml'))

In [ ]:
# ov_post_quant_conv = mo.convert_model(str(AUTOENCODER_PATH / 'post_quant_conv.onnx'))
# serialize(ov_post_quant_conv, str(AUTOENCODER_PATH / 'post_quant_conv.xml'))

In [ ]:
model, _, _ = open_clip.create_model_and_transforms(
            'ViT-H-14', device=torch.device('cpu'), pretrained=os.path.join(p.model.model_dir, p.model.config.model.model_args.ckpt_clip))

In [ ]:
import torch.nn.functional as F
from functools import partial

def forward(self, x: torch.Tensor):
    print(x.shape)
    raise
    orig_type = x.dtype
    x = F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
    return x.to(orig_type)
p.model.clip_encoder.model.ln_final.forward = partial(forward, p.model.clip_encoder.model.ln_final)

In [ ]:
def generate(text):
    output_video_path = p({'text': text}, output_video='./output.mp4', device='cpu')[OutputKeys.OUTPUT_VIDEO]
    return output_video_path

demo = gr.Interface(
    generate,
    gr.Textbox(),
    gr.Video(),
    examples=['A panda eating bamboo on a rock.']
)
demo.queue().launch(share=True, debug=True)

In [ ]:
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

video = torch.randn()

In [ ]:
video.shape

NameError: name 'video' is not defined

In [ ]:
type(p.model.sd_model)

openvino.runtime.ie_api.CompiledModel